In [65]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, classification_report
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor, XGBClassifier

import warnings


In [2]:
data = pd.read_csv("data_file/loan-test.csv")

In [3]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [4]:
data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,367.000000,367.000000,362.000000,361.000000,338.000000
mean,4805.599455,1569.577657,136.132597,342.537396,0.825444
std,4910.685399,2334.232099,61.366652,65.156643,0.380150
min,0.000000,0.000000,28.000000,6.000000,0.000000
25%,2864.000000,0.000000,100.250000,360.000000,1.000000
50%,3786.000000,1025.000000,125.000000,360.000000,1.000000
75%,5060.000000,2430.500000,158.000000,360.000000,1.000000
max,72529.000000,24000.000000,550.000000,480.000000,1.000000


In [41]:
X=data.drop(columns=['Loan_ID'],axis=1, errors='ignore')

In [7]:
X.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [8]:
# Displaying unique categories for each categorical variable

print("Categories in 'Gender' variable:  ",end=" ")
print(data['Gender'].unique())

print("Categories in'Married' variable:  ", end=" ")
print(data['Married'].unique())

print("Categories in 'Education' variable:  ", end=" ")
print(data['Education'].unique())

print("Categories in 'Self_Employed' variable:  ", end=" ")
print(data['Self_Employed'].unique())

print("Catogories in 'Peoperty_Area' variable: ", end=" ")
print(data["Property_Area"].unique())

Categories in 'Gender' variable:   ['Male' 'Female' nan]
Categories in'Married' variable:   ['Yes' 'No']
Categories in 'Education' variable:   ['Graduate' 'Not Graduate']
Categories in 'Self_Employed' variable:   ['No' 'Yes' nan]
Catogories in 'Peoperty_Area' variable:  ['Urban' 'Semiurban' 'Rural']


In [15]:
# Handling the missing values

# Fill categorical columns with mode
categorical_cols = ['Gender', 'Dependents', 'Self_Employed', 'Credit_History']
for col in categorical_cols:
    data[col].fillna(data[col].mode()[0], inplace=True)

# Fill numerical columns 
data['LoanAmount'].fillna(data['LoanAmount'].median(),inplace=True)
data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mode()[0], inplace=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_1140\1604466376.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['LoanAmount'].fillna(data['LoanAmount'].median(),inplace=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_1140\1604466376.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [27]:
X = data.drop('Credit_History', axis=1)
Y = data['Credit_History']

In [29]:
Y

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
362    1.0
363    1.0
364    1.0
365    1.0
366    1.0
Name: Credit_History, Length: 367, dtype: float64

In [ ]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [21]:
# Create Column Transformer with 3 types of transformers
num_features=X.select_dtypes(exclude="object").columns
cat_features=X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features), 
         ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [22]:
X = preprocessor.fit_transform(X)

In [26]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4037 stored elements and shape (367, 386)>

In [30]:
#seperate the data in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((293, 11), (74, 11))

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score

def evaluate_model(true, predicted):
    # For classification, we use Accuracy, F1, Precision, or Recall
    accuracy = accuracy_score(true, predicted)
    f1 = f1_score(true, predicted)
    precision = precision_score(true, predicted)
    return accuracy, f1, precision # Return three classification metrics

In [52]:
X = data.drop(['Loan_ID',], axis=1, errors='ignore')

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree Classifier": DecisionTreeClassifier(random_state=42),
    "Random Forest Classifier": RandomForestClassifier(random_state=42),
    "XGB Classifier": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "CatBoost Classifier": CatBoostClassifier(verbose=False, random_state=42)
}

model_list = []
# Change this list name to store a classification metric, e.g., 'f1_list'
r2_list =[] 

for i in range(len(list(models))):
    model = list(models.values())[i]
    
    # --- ASSUMPTION: X_train, y_train, X_test, y_test are already defined ---
    # You must have split your training data into these four sets.
    
    # Train model
    model.fit(X_train, y_train) 

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset (Note: Metrics are classification metrics now)
    # The variable names here still use the old regression names (mae, rmse, r2) for structure,
    # but they now represent classification metrics (e.g., Accuracy, F1-Score, Precision).
    model_train_metric1 , model_train_metric2, model_train_metric3 = evaluate_model(y_train, y_train_pred)

    model_test_metric1 , model_test_metric2, model_test_metric3 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    # IMPORTANT: Update the print labels to match your new classification metrics (e.g., Accuracy, F1 Score)
    print('Model performance for Training set')
    print("- Metric 1 (e.g., Accuracy): {:.4f}".format(model_train_metric1))
    print("- Metric 2 (e.g., F1 Score): {:.4f}".format(model_train_metric2))
    print("- Metric 3 (e.g., Precision): {:.4f}".format(model_train_metric3)) # Was R2 Score

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Metric 1 (e.g., Accuracy): {:.4f}".format(model_test_metric1))
    print("- Metric 2 (e.g., F1 Score): {:.4f}".format(model_test_metric2))
    print("- Metric 3 (e.g., Precision): {:.4f}".format(model_test_metric3)) # Was R2 Score
    r2_list.append(model_test_metric3) # Add one test metric to the list
    
    print('='*35)
    print('\n')


Logistic Regression
Model performance for Training set
- Metric 1 (e.g., Accuracy): 0.8430
- Metric 2 (e.g., F1 Score): 0.9145
- Metric 3 (e.g., Precision): 0.8425
----------------------------------
Model performance for Test set
- Metric 1 (e.g., Accuracy): 0.8378
- Metric 2 (e.g., F1 Score): 0.9118
- Metric 3 (e.g., Precision): 0.8378


Decision Tree Classifier
Model performance for Training set
- Metric 1 (e.g., Accuracy): 1.0000
- Metric 2 (e.g., F1 Score): 1.0000
- Metric 3 (e.g., Precision): 1.0000
----------------------------------
Model performance for Test set
- Metric 1 (e.g., Accuracy): 0.7973
- Metric 2 (e.g., F1 Score): 0.8837
- Metric 3 (e.g., Precision): 0.8507


Random Forest Classifier
Model performance for Training set
- Metric 1 (e.g., Accuracy): 1.0000
- Metric 2 (e.g., F1 Score): 1.0000
- Metric 3 (e.g., Precision): 1.0000
----------------------------------
Model performance for Test set
- Metric 1 (e.g., Accuracy): 0.8243
- Metric 2 (e.g., F1 Score): 0.9037
- Metr

d:\LOAN ELIGIBILITY\myvenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:48:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGB Classifier
Model performance for Training set
- Metric 1 (e.g., Accuracy): 1.0000
- Metric 2 (e.g., F1 Score): 1.0000
- Metric 3 (e.g., Precision): 1.0000
----------------------------------
Model performance for Test set
- Metric 1 (e.g., Accuracy): 0.8108
- Metric 2 (e.g., F1 Score): 0.8923
- Metric 3 (e.g., Precision): 0.8529


CatBoost Classifier
Model performance for Training set
- Metric 1 (e.g., Accuracy): 0.9454
- Metric 2 (e.g., F1 Score): 0.9685
- Metric 3 (e.g., Precision): 0.9389
----------------------------------
Model performance for Test set
- Metric 1 (e.g., Accuracy): 0.8378
- Metric 2 (e.g., F1 Score): 0.9104
- Metric 3 (e.g., Precision): 0.8472




In [53]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
3,XGB Classifier,0.852941
1,Decision Tree Classifier,0.850746
4,CatBoost Classifier,0.847222
0,Logistic Regression,0.837838
2,Random Forest Classifier,0.835616


In [61]:
from sklearn.metrics import accuracy_score

lin_model = LogisticRegression(max_iter=1000)
lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = accuracy_score(y_test, y_pred) * 100
print(f"Accuracy of the model is {score:.2f}%")


Accuracy of the model is 83.78%


### THE BEST MODEL HERE IS LOGISTIC 

In [67]:


# Train final model
xgb_model = XGBClassifier(
    n_estimators=200, learning_rate=0.1,
    max_depth=4, subsample=0.8,
    colsample_bytree=0.8, random_state=42,
    use_label_encoder=False, eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = xgb_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8378378378378378


d:\LOAN ELIGIBILITY\myvenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:09:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
